# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

In [1]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. 

In [2]:
###### Test pierwszości Rabina-Millera
import random

def rabinMiller(num):
    d = num - 1    ##obliczamy wartości d i sa
    s = 0
    while d % 2 == 0:   ##usuwamy z num-1 dzielniki 2 zliczając je w s
        d = d // 2
        s += 1

    for trials in range(6):   ## wykonujemy n testów R-B 
        x = random.randrange(2, num - 1)  ##losujemy baze x
        a = pow(x, d, num)   ### pierwszy wyraz ciągu
        if (a != 1) and (a != num-1): ## jeśli a nie spełnia warunków losujemy innego świadka
            i = 1
            while (i < s) and (a != (num - 1)): #generujemy ciąg R-M
                a = (a ** 2) % num   ## obliczamy kolejne wyrazy ciągu R-M: b^d, b^2d, b^4d
                if(a==1):          ## tylko ostatni wyraz może mieć wartość 1! skończyliśmy za wcześnie
                    return False ##To nie jest liczba pierwsza lub silnie pseudopierwsza
                i+=1
            if(a!=num-1):  ##przedostatni wyraz musi mieć wartość num-1    
                return False  ##To nie jest liczba pierwsza lub silnie pseudopierwsza                
    ### jeśli wykonaliśmy n testów i żaden nie zakończył się False 
    return True

def isPrime(num):
    if (num <= 2):
        return False # oczywista oczywistość 
    #opcjonalne można sprawdzić czy małe liczby pierwsze nie są czynnikami badanej liczby   
    return rabinMiller(num)


isPrime(65537)

True

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [3]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    while True:
        num = random.randrange(2**(keysize-1),2**keysize)
        if isPrime(num):
            return num
    
generatePrime(1000)

6333543989942971715012110347597915886895526269818107766564015954907934575927090340088579638308165593037160350420941015720762423862986288212545837304039919400279279195658024955233365910545573997344265485280456202813881206861538185588561309373687686199014942583713274050936602218849883553319402634802447

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [4]:
import random, sys, os

def generateKey(keySize):
    p = generatePrime(keySize)
    q = generatePrime(keySize)
    n = p * q

    while True:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))
        if gcd(e, (p - 1) * (q - 1)) == 1:
             break
                
    d = findModInverse(e, (p - 1) * (q - 1))
    publicKey = (n, e)
    privateKey = (n, d)
    print('Public key:', publicKey)
    print('Private key:', privateKey)

    return (publicKey, privateKey)


def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
  
print('Generujemy klucze publiczny i prywatny')
makeKeyFiles(16)

Generujemy klucze publiczny i prywatny
Public key: (1566568483, 61301)
Private key: (1566568483, 1339850077)


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [5]:
def encrypt(message, modulus, exp):
    # kod szyfrowania
    message_encrypted = []
    for letter in message:
        message_encrypted += (ord(letter) ** exp) % modulus
  
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [6]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = []
    for number in message_encrypted:
        message_ascii += chr((number ** exp) % modulus)
        
    return ('').join(message_ascii)



IndentationError: unexpected indent (<ipython-input-6-6fd671d23c6d>, line 3)

## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie 
Zademonstruj deformowalność podręcznikowego szyfrowania RSA